In [1]:
%%html
<style>
table {
    display: inline-block
}
</style>

# 数据库：MySQL&MariaDB
> <img style="width: 100px; display: inline-block; " src="https://resource.xtalker.cn:8000/_static/img/mysql-logo.png"/>
> <br><img style="width: 100px; display: inline-block; " src="https://resource.xtalker.cn:8000/_static/img/mariadb-logo.png" />

    MySQL：由瑞典MySQL AB公司开发，后来被SUN公司收购，再后来被甲骨文公司（Oracle）收购。
    MariaDB：是MySQL的一个分支，主要由开源社区在维护，由MySQL的创始人Michael Widenius主导开发。MariaDB的名称来自Michael Widenius的女儿Maria的名字。

### 贴士：名字和Logo
        据信MySQL和MariaDB名字都与创始人Michael Widenius女儿的名字有关。
        MySQL的海豚标志的名字叫“sakila”，它是由MySQL AB的创始人从用户在“海豚命名”的竞赛中建议的大量的名字表中选出的。获胜的名字是由来自非洲斯威士兰的开源软件开发者Ambrose Twebaze提供。根据Ambrose所说，Sakila来自一种叫SiSwati的斯威士兰方言，也是在Ambrose的家乡乌干达附近的坦桑尼亚的Arusha的一个小镇的名字。
        MariaDB的标志没有出处。
    本章内容：学习MariaDB官方镜像

### 贴士：兼容性
        Mysql5和MariaDB10保持高度的版本兼容性，几乎你用到的各种工具都是可以互换使用的，而之后的版本各自发展，并不一定保持一致。

### 贴士：管理工具
> <img style="width: 100px; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/phpMyAdmin-logo.png"/>

        Mysql和MariaDB不仅客户端Cli管理工具保持高度兼容，图形化管理工具：phpMyAdmin也保持高度兼容。

## 目录

1. [概述](#1.概述)
    <br> &emsp; [1.1.组网原理](#1.1.组网原理)
    <br> &emsp; [1.2.Bridge网络模型](#1.2.Bridge网络模型)
    <br> &emsp; [1.3.镜像版本](#1.3.镜像版本)

2. [快速开始](#2.快速开始)
    <br> &emsp; [镜像下载](#镜像下载)
    <br> &emsp; [Bridge网络](#Bridge网络)
    <br> &emsp; [镜像运行db1：mysql](#镜像运行db1：mysql)
    <br> &emsp; [镜像运行db2：mariadb](#镜像运行db2：mariadb)
    <br> &emsp; [镜像运行admin：phpMyAdmin](#镜像运行admin：phpMyAdmin)
    <br> &emsp; [使用](#使用)
    <br> &emsp; [清理](#清理)
    <br> &emsp; [操作过程](#操作过程)


## 参考资料

名称 | 链接 | 说明
:-- | :-- | :--
MySQL | https://www.mysql.com/ | 官网
MariaDB | https://mariadb.org/ | 官网
MariaDB | https://www.phpmyadmin.net/ | 官网


# 1.概述
## 1.1.组网原理
> <img style="zoom:80%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/数据库和应用组网图.png"/>

    宿主机的浏览器充当客户端；PhpMyAdmin充当应用服务器；而MySQL|MariaDB提供数据库服务。
    从组网图可以看出，PhpMyAdmin和MySQL必须置于同一个局域网中才能相互通信；而宿主机上的浏览器能够访问容器必须通过端口映射。

## 1.2.Bridge网络模型
> <img style="zoom:50%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/docker_Bridge网络模型.png"/>


## 1.3.镜像版本
    Mysql官方版本：
        docker pull mysql:5.7
    MariaDB官方版本：
        docker pull mariadb:10
    选择因素：
        Mysql 5.7 和 MariaDB 10保持高度兼容性，如果不追去最新功能特性，可以选择这两个版本
    phpMyAdmin官方版本：
        docker pull phpmyadmin:latest


# 2.快速开始

### 镜像下载
```shell
docker pull mysql:5.7
docker pull mariadb:10
docker pull phpmyadmin:latest
------------------------------
...

docker images
------------------------------
REPOSITORY              TAG             IMAGE ID       CREATED        SIZE
phpmyadmin              latest          87501b683e84   12 days ago    562MB
mariadb                 10              9200bd86da81   2 months ago   404MB
mysql                   5.7             5107333e08a8   4 months ago   501MB
```

### Docker默认创建的网络对象

```shell
# 列表
docker network ls
NETWORK ID     NAME        DRIVER    SCOPE
f81f774df348   bridge      bridge    local
803cca1cd69a   host        host      local
8442ae8d48c4   none        null      local

# 详情
docker network inspect bridge
[
    {
        "Name": "bridge",
        "Id": "f81f774df348f67c86c2414b06bcba41ff30a600583702d92ad3338ff2a1ad4c",
        "Created": "2024-05-07T09:50:05.89794715Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/24",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {
            "com.docker.network.bridge.default_bridge": "true",
            "com.docker.network.bridge.enable_icc": "true",
            "com.docker.network.bridge.enable_ip_masquerade": "true",
            "com.docker.network.bridge.host_binding_ipv4": "0.0.0.0",
            "com.docker.network.bridge.name": "docker0",
            "com.docker.network.driver.mtu": "1500"
        },
        "Labels": {}
    }
]
```
> Docker安装后首次运行至少存在以上三个网络，其中bridge网络是bridge类型默认创建网络。

### 创建Bridge类型的网络：取个名字叫做 db-br

```shell
docker network create db-br
05cab70a3e2458044ca26a9f9294613fc710b326dcf3d0751f067d84b102b20b

C:\Btsync\teacher\docker>docker network list
NETWORK ID     NAME        DRIVER    SCOPE
f81f774df348   bridge      bridge    local
05cab70a3e24   db-br       bridge    local
803cca1cd69a   host        host      local
8442ae8d48c4   none        null      local
```

### 镜像运行db1：mysql

```shell
# 运行
docker run --rm -it -d --name db1 --hostname db1 -e MYSQL_ROOT_PASSWORD=my-password --network db-br mysql:5.7
04d8ede446e0096b1df0f4596f88b278c516d025ee23a3554ec66fcd461aaf76

# 进入容器 shell
docker exec -it db1 bash
bash-4.2# 

# 执行mysql cli登录指令
mysql -u root -p
Enter password:
Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 5
Server version: 5.7.44 MySQL Community Server (GPL)

Copyright (c) 2000, 2023, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql>
```
> 当出现Linux命令提示符号 `mysql>` 的时候，说明你已经可以操作数据库了。


### 镜像运行db2：mariadb
    所有操作与db1几乎一样，只是使用的镜像不同。注意分别取不同的名字以示区分。
```shell
# 运行
docker run --rm -it -d --name db2 --hostname db2 -e MYSQL_ROOT_PASSWORD=my-password --network db-br mariadb:10
6e8980d045cc1e18ece662c363b0cfd77483916ef73ee9f606c8f5e726bedf60

# 进入容器 shell
docker exec -it db2 bash
root@db2:/#

# 执行mysql cli登录指令
mysql -u root -p
Enter password:
Welcome to the MariaDB monitor.  Commands end with ; or \g.
Your MariaDB connection id is 6
Server version: 10.11.7-MariaDB-1:10.11.7+maria~ubu2204 mariadb.org binary distribution

Copyright (c) 2000, 2018, Oracle, MariaDB Corporation Ab and others.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

MariaDB [(none)]>
```
> 当出现Linux命令提示符号 `MariaDB [(none)]>` 的时候，说明你已经可以操作数据库了。

### 镜像运行admin：phpMyAdmin
    这是一个管理数据库的应用。我们通过将数据库和管理器分配到同一个网络中，实现管理器和数据库的网络直达。
```shell
# 运行
docker run --rm -it -d --name admin --hostname admin -e PMA_ARBITRARY=1 --network db-br -p 80:80 phpmyadmin
9af05c7fe6448131a71e6f8edefe0a996ceb5eaa990bc35bf40b4a800059bf9b
```
> 打开你的浏览器，输入：http://localhost 网址，是否出现如下的界面呢？
<br><img style="zoom:50%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/phpMyAdmin-welcome.png"/>


### 网络测试

```shell
# 首先验证测试一下网络是否通畅，我们需要一个工具
docker pull busybox
----------------------
...
# 使用 busybox
docker run -it --rm --network db-br busybox

ping db1 # ctrl + c 退出命令
PING db1 (172.17.2.2): 56 data bytes
64 bytes from 172.17.2.2: seq=0 ttl=64 time=0.370 ms
64 bytes from 172.17.2.2: seq=1 ttl=64 time=0.072 ms
64 bytes from 172.17.2.2: seq=2 ttl=64 time=0.109 ms
^C
--- db1 ping statistics ---
3 packets transmitted, 3 packets received, 0% packet loss
round-trip min/avg/max = 0.072/0.183/0.370 ms

ping db2 # ctrl + c 退出命令
PING db2 (172.17.2.3): 56 data bytes
64 bytes from 172.17.2.3: seq=0 ttl=64 time=0.292 ms
64 bytes from 172.17.2.3: seq=1 ttl=64 time=0.194 ms
^C
--- db2 ping statistics ---
2 packets transmitted, 2 packets received, 0% packet loss
round-trip min/avg/max = 0.194/0.243/0.292 ms

ping admin # ctrl + c 退出命令
PING admin (172.17.2.4): 56 data bytes
64 bytes from 172.17.2.4: seq=0 ttl=64 time=0.546 ms
64 bytes from 172.17.2.4: seq=1 ttl=64 time=0.111 ms
^C
--- admin ping statistics ---
2 packets transmitted, 2 packets received, 0% packet loss
round-trip min/avg/max = 0.111/0.328/0.546 ms
```
    通过busybox工具我们知道db1/db2/admin容器的主机名和IP地址，并且网络是通畅的。
    我们可以愉快的在浏览器登录界面中输入：
        服务器：db1 或者 db2
        用户名：root
        密码：你设置的数据库root用户密码

### 清理
    很多同学不清楚如何清除实验过程和结果，导致再次测试验证时出错。
        docker stop db1
        docker stop db2
        docker stop admin
        docker network rm db-br
        # 如果要清除所有容器/镜像和网络等等资源，谨慎操作
        docker system prune -a

### 操作过程
> [![asciicast](https://resource.xtalker.cn:8000/_static/cast/mysql.svg)](https://asciinema.xtalker.cn:8000/a/9)
<br> <a href="https://asciinema.xtalker.cn:8000/a/9" target="_blank">实验十 Docker应用之数据库</a>